In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [2]:
tokenizer = AutoTokenizer.from_pretrained("NlpHUST/ner-vietnamese-electra-base")
model = AutoModelForTokenClassification.from_pretrained("NlpHUST/ner-vietnamese-electra-base")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

In [3]:
f = open('text.txt','r')
lines = f.readlines()
f.close()

In [9]:
def post_process(text):
    text = text.replace(' ##','')
    return text
def predict(text):
    ner_results = nlp(text)
    ner_results = [i for i in ner_results if i['score']>0.5]
    merged_entities = []
    current_entity = None
    for entity in ner_results:
        if entity['entity'].startswith('B-'):
            if current_entity:
                merged_entities.append(current_entity)
            current_entity = entity
        elif entity['entity'].startswith('I-'):
            if current_entity and current_entity['entity'][2:] == entity['entity'][2:]:
                current_entity['word'] += ' ' + entity['word']
                current_entity['end'] = entity['end']
            else:
                if current_entity:
                    merged_entities.append(current_entity)
                current_entity =entity
        else:
            if current_entity:
                merged_entities.append(current_entity)
            current_entity = None

    if current_entity:
        merged_entities.append(current_entity)

    keys = ['PERSON','LOCATION']
    results = {}
    for key in keys:
        results[key] = []
    for entity in merged_entities:
        entity_type = entity['entity'].split('-')[1]
        if entity_type in keys:
            results[entity_type].append(post_process(entity['word']))
    return results

In [10]:
for i in range(len(lines)):
    print(lines[i])
    print(predict(lines[i]))
    print('-------------')
    if i==10: break

Chương 1. Kẻ thù một mất một còn.

{'PERSON': [], 'LOCATION': []}
-------------
“Ngươi mới chính là tân Đế Đại Ân.”

{'PERSON': [], 'LOCATION': ['Đại']}
-------------
…

{'PERSON': [], 'LOCATION': []}
-------------
Đất nước sụp đổ.

{'PERSON': [], 'LOCATION': []}
-------------
Mây đen che phủ thành Nghiệp Kinh cũng chẳng thể đè ép được chiến hỏa thiêu đốt hừng hực một đường từ biên giới tới nội cung. Mấy trăm ngàn Khải Phong binh và phản thần nội ứng ngoại hợp, suốt đêm đánh thẳng vào cung đình Nghiệp Kinh.

{'PERSON': [], 'LOCATION': ['thành Nghiệp Kinh', 'Nghiệp Kinh']}
-------------
Đêm ấy, người kêu ngựa hí, ba ngàn cấm quân thất thủ không chỗ bại lui, nhóm cung nhân chưa kịp thu thập châu báu đã dồn dập chạy trối chết.

{'PERSON': [], 'LOCATION': []}
-------------
Ân Đế Lâm Nghiêu cùng đường mạt lộ, bị bức ép thắt cổ tự vẫn tại Trường Minh điện.

{'PERSON': ['Ân Đế Lâm Nghiêu'], 'LOCATION': ['Trường Minh điện']}
-------------
“Bạo Ân tất giết, Đại Khải lên ngôi!” “Bạo Ân tất giết,